# Visualising Emissions

It is often useful to be able to produce visualisations of profiling data, including CO~2~ estimates.

Here we will use the [Weights & Biases](https://wandb.ai) platform to construct a simple profiling dashboard for our code.

### Step 1: 
Visit https://wandb.ai/authorize

### Step 2: 
Log in with your GitHub account or with Google

### Step 3: 
Copy API key (under the profile dropdown on the right-hand side)

### Step 4: 
Log in to W&B 

- In VSCode Terminal, run python3 (or python)

- `import wandb`

- `wandb.login()`

- Enter your choice: 2

- Paste the copied API key


### Step 5: 
Exit python environment from terminal

### Step 6: 
Run your script with `wandb.init()` and then `wandb.log`, and `wandb.finish()` to end the wandb run.

In [ ]:
from codecarbon import EmissionsTracker
import pandas as pd
import wandb


# Function to retrieve desired stats from file emissions.csv
def get_stats():
    data = pd.read_csv('emissions.csv').iloc[-1,:] # get the last CSV row as a Series
    return {
        "duration": data['duration'],
        "emissions": data['emissions'],
        "cpu_power": data['cpu_power'],
        "gpu_power": data['gpu_power'],
        "ram_power": data['ram_power']
    }

# Function to get run multiple iterations and log the stats
def run_and_log(project_name, analysis_func, num_runs):
    results = []
    
    # Initialize wandb run
    wandb.init(project=project_name, reinit=True)

    try:
        for run_idx in range(num_runs):

            tracker = EmissionsTracker()
            tracker.start()
            
            try:            
                result = analysis_func()

            except Exception as e:
                print(f"An error occurred: {e}")

            finally:
                emissions: float = tracker.stop()

            results.append(result)

            # Log per-run metrics to wandb
            wandb.log(get_stats())

            # Print result
            print(result)

    except Exception as e:
        print(f"An error occurred: {e}")

    finally:
        # End wandb run
        wandb.finish()  
    
    return (results)

In [ ]:
def fib(n):
    if(n == 0):
        return 0
    if(n == 1):
        return 1
    return fib(n - 1) + fib(n - 2)

def fibs(n):
    return [fib(i) for i in range(n)]

def fib_task():
    return fibs(35)


In [ ]:

results = run_and_log("fib", fib_task, 10)

print(results[0])


## Task: C0~2~ visualisation

Try making a W&B dashboard for your pathfinding code.

In [ ]:
# (from solutions to notebook 01)
import networkx as nx

# function to find all the possible extensions of a path
def extend(G, path):
    ex = []
    for v in G.neighbors(path[-1]):
        if v not in path:
            ex.append( path + [v] )
    return ex

# brute force pathfinding
def path_brute(G, start, end):

    # initialise paths
    P = [ [start] ]   # stores all candidate paths originating at `start`

    # loop
    finished = False
    while(not finished):
        finished = True
        P_next = []
        for p in P:
            ex = extend(G, p)
            for p2 in ex:
                
                if p2[-1] == end:
                    return p2                  # we have a solution!
                else:
                    finished = False           # this path needs to be processed further
                    P_next.append(p2)
        P = P_next


In [ ]:
# Load the data
file_path = "data/facebook.txt"
G = nx.read_adjlist(file_path, nodetype=int)

# task to be profiled
def task_brute():
    return path_brute(G, 0, 4038)

In [ ]:
results = run_and_log("brute", task_brute, 10)

print(results[0])

---